In [70]:
import torch
import sklearn.datasets as skds
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
import torchvision
from torch import nn
import numpy
import pandas
from typing import Optional
from tqdm import tqdm

# Q1 A - basic code

In [71]:
x = torch.arange(0, 10, dtype=torch.float)

mask = torch.bernoulli(x, 0.5)  # 0 and 1 distrobution
# negative_mask = torch.tensor((mask == 0), dtype=torch.uint8)  # Opposite of mask

# Y = mask * a  # Apply mask 
# Y

mu = x.mean() # Mean
sigma = x.std() # Standard deviation
epsilon = .00001 # To avoid division by zero

a = torch.sqrt((sigma ** 2) + epsilon)

calc = (x - mu) / a 

print("x: \n", x)
print("x-mu: \n", x-mu)
print("(x-mu)/a: \n", (x-mu)/a)
print("mask: \n", mask)

x = torch.mul(calc, mask)



print("after mask and calculations: \n", x)

x: 
 tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])
x-mu: 
 tensor([-4.5000, -3.5000, -2.5000, -1.5000, -0.5000,  0.5000,  1.5000,  2.5000,
         3.5000,  4.5000])
(x-mu)/a: 
 tensor([-1.4863, -1.1560, -0.8257, -0.4954, -0.1651,  0.1651,  0.4954,  0.8257,
         1.1560,  1.4863])
mask: 
 tensor([1., 0., 0., 0., 0., 1., 0., 1., 0., 1.])
after mask and calculations: 
 tensor([-1.4863, -0.0000, -0.0000, -0.0000, -0.0000,  0.1651,  0.0000,  0.8257,
         0.0000,  1.4863])


In [72]:
class DropNorm(nn.Module):
	def __init__(self, size_in, size_out):
		super().__init__()

		self.size_in, self.size_out = size_in, size_out

		self.weights = nn.Parameter(torch.Tensor(size_out, size_in))  # nn.Parameter is a Tensor that's a module parameter.
		self.bias = nn.Parameter(torch.Tensor(size_out))

		self.linear1 = nn.Linear(size_in, size_out)
		self.activation = nn.ReLU(inplace=True) # inplace = don't use extra memory

		self.drop_probability = 0.5  # We can take as argument but this question doesn't requires it.
		
	
	def forward(self, batch):
		# First dim = batch size
		# Second dim = sample

		mu = batch.mean(dim=1) # Mean
		sigma = batch.std(dim=1) # Standard deviation
		epsilon = .00001 # To avoid division by zero
		Y = torch.sqrt((sigma ** 2) + epsilon)

		# batch: [600, 2049]
		# mu: [600]
		# We need to substract mu from each dimension of batch.

		# (batch[0] - mu[0]) is like: [2049] - [1]

		#batch[0] = batch[0] - mu[0]
		#batch[1] = batch[1] - mu[1]
		# ...

		new_x = (batch - mu) / Y

		return new_x

model = nn.Sequential(
	nn.Linear(784, 2049),
	nn.ReLU(),

	DropNorm(2049, 2049),
	nn.ReLU(),

	nn.Linear(2049, 20),
	nn.Softmax()
)

# model = nn.Sequential(
# 	DropNorm(784, 784),

# 	nn.Linear(784, 2049),
# 	nn.ReLU(),

# 	nn.Linear(2049, 512),
# 	nn.ReLU(),

# 	nn.Linear(512, 20),
# 	nn.Softmax()
# )

In [73]:
batch_size = 600


dataset = torchvision.datasets.FashionMNIST(root="/22961", train=True, download=True, transform=torchvision.transforms.PILToTensor())
dataloader = DataLoader(dataset, batch_size=batch_size)

print("num of samples:", len(dataset))
batches=len(dataloader)
print("batch size: ", batch_size)
print("num of batches:", batches)
print("image size:", dataset[0][0].shape)

num of samples: 60000
batch size:  600
num of batches: 100
image size: torch.Size([1, 28, 28])


In [74]:
loss_func=torch.nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [75]:
def iterate_batch(idx):
	imgs, labels = next(iter(dataloader))
	imgs = imgs.flatten(start_dim=1)

	optimizer.zero_grad()
	y_model=model(imgs.float())

	loss=loss_func(y_model,labels)
	loss.backward()
	optimizer.step()

	predicted_labels = y_model.argmax(dim=1)
	acc = (predicted_labels == labels).sum()/len(labels)
	return loss.detach(), acc.detach()

In [76]:
# Computing on GPU (if possible)
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

get_default_device()

device(type='cpu')

In [78]:
batch_loss=torch.zeros(batches)
batch_acc=torch.zeros(batches)
for idx in tqdm(range(batches)):
	batch_loss[idx], batch_acc[idx] = iterate_batch(idx)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(range(1,batches+1), batch_loss,label="Train Loss")
plt.title("Loss")
plt.legend()
plt.xlabel("Epoch Number")